In [1]:
# Setup: install timm (respect existing torch), load JSONs, build label mapping, stratified split
import os, sys, json, math, random, time, shutil, subprocess
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.backends.cudnn as cudnn

def run(cmd):
    print('>', ' '.join(cmd), flush=True)
    subprocess.run(cmd, check=True)

def pip_install_pkgs():
    # Freeze torch stack; avoid re-installing torch/torchvision
    cons = Path('constraints.txt')
    if not cons.exists():
        cons.write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')
    # Install timm without deps to prevent torch reinstallation
    run([sys.executable, '-m', 'pip', 'install', 'timm==1.0.9', '--no-deps'])
    # Install required deps including HF hub BEFORE importing timm so timm sees them
    run([sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', 'albumentations', 'opencv-python-headless', 'huggingface_hub', 'safetensors', '--upgrade-strategy', 'only-if-needed'])

print('torch:', torch.__version__, 'cuda build:', getattr(torch.version,'cuda', None), 'cuda avail:', torch.cuda.is_available(), flush=True)
pip_install_pkgs()
import timm

cudnn.benchmark = True
torch.set_float32_matmul_precision('high')
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU', flush=True)

# Load JSONs
def load_json(fp):
    with open(fp, 'r') as f:
        return json.load(f)
train_js = load_json('train2019.json')
test_js = load_json('test2019.json')

# Build dataframe from train JSON using JSON-based category_id
train_images = {im['id']: im for im in train_js['images']}
rows = []
for ann in train_js['annotations']:
    im = train_images.get(ann['image_id'])
    if im is None: continue
    rows.append({
        'image_id': ann['image_id'],
        'file_name': im['file_name'],
        'category_id': ann['category_id']
    })
df = pd.DataFrame(rows)
print('Train rows:', len(df), 'unique cats:', df['category_id'].nunique(), flush=True)

# Label mapping: map arbitrary category_id -> contiguous [0..C-1]
cat_ids = sorted(df['category_id'].unique().tolist())
cat_id_to_idx = {cid:i for i,cid in enumerate(cat_ids)}
idx_to_cat_id = {i:cid for cid,i in cat_id_to_idx.items()}
df['label'] = df['category_id'].map(cat_id_to_idx)
num_classes = len(cat_ids)
print('num_classes:', num_classes, 'min/max cat_id:', min(cat_ids), max(cat_ids), flush=True)

# Stratified split from train (since official val images are not present in extracted files)
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.0115, random_state=42)  # ~3030 holdout to mirror official val size
train_idx, val_idx = next(sss.split(df.index.values, df['label'].values))
df_train = df.iloc[train_idx].reset_index(drop=True)
df_val = df.iloc[val_idx].reset_index(drop=True)
print('Split -> train:', len(df_train), 'val:', len(df_val), flush=True)

# Save mappings for reuse
Path('artifacts').mkdir(exist_ok=True)
pd.Series(idx_to_cat_id).to_json('artifacts/idx_to_cat_id.json')
pd.Series(cat_id_to_idx).to_json('artifacts/cat_id_to_idx.json')
df_train.to_csv('artifacts/train_split.csv', index=False)
df_val.to_csv('artifacts/val_split.csv', index=False)
print('Prepared splits and mappings.', flush=True)

torch: 2.4.1+cu121 cuda build: 12.1 cuda avail: True


> /usr/bin/python3.11 -m pip install timm==1.0.9 --no-deps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.2 MB/s eta 0:00:00


> /usr/bin/python3.11 -m pip install -c constraints.txt albumentations opencv-python-headless huggingface_hub safetensors --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 KB 13.3 MB/s eta 0:00:00


...

GPU: NVIDIA A10-24Q


Train rows: 232999 unique cats: 1010


num_classes: 1010 min/max cat_id: 0 1009


Split -> train: 230319 val: 2680


Prepared splits and mappings.


In [6]:
# Dataset, transforms, model, and quick smoke-train to validate pipeline (torchvision backbone to avoid HF issues)
import math, time, gc, os
from pathlib import Path
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import torchvision.models as tvm
from torchvision.transforms import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class JsonImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img_path = row['file_name']
        with Image.open(img_path) as img:
            img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        label = int(row['label'])
        return img, label

# Torchvision ResNet50 (URL-hosted weights) to bypass HF hub
weights = tvm.ResNet50_Weights.IMAGENET1K_V2
print('Creating model: torchvision resnet50, weights IMAGENET1K_V2', flush=True)
base_model = tvm.resnet50(weights=weights)
in_feats = base_model.fc.in_features
base_model.fc = nn.Linear(in_feats, num_classes)
model = base_model.to(device).to(memory_format=torch.channels_last)

# Transforms (use standard ImageNet mean/std to avoid weights.meta dependency)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
img_size = 224
train_tfms = T.Compose([
    T.RandomResizedCrop(img_size, scale=(0.5, 1.0), interpolation=InterpolationMode.BICUBIC),
    T.RandomHorizontalFlip(p=0.5),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])
val_tfms = T.Compose([
    T.Resize(int(img_size/0.875), interpolation=InterpolationMode.BICUBIC),
    T.CenterCrop(img_size),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])

# Datasets and loaders (small subsets for smoke test)
n_train_smoke = 2048
n_val_smoke = 512
train_ds_full = JsonImageDataset(df_train, transform=train_tfms)
val_ds_full = JsonImageDataset(df_val, transform=val_tfms)
train_ds = Subset(train_ds_full, np.arange(min(n_train_smoke, len(train_ds_full))))
val_ds = Subset(val_ds_full, np.arange(min(n_val_smoke, len(val_ds_full))))

bs = 96
nw = min(8, os.cpu_count() or 4)
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=nw, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=nw, pin_memory=True, persistent_workers=True)

# Optim, loss
lr = 1e-3 * (bs / 256.0)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    t0 = time.time()
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True, memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(xb)
            pred = logits.argmax(dim=1)
            correct += (pred == yb).sum().item()
            total += yb.numel()
    acc = correct / max(1, total)
    print(f'Eval: acc={acc:.4f}, n={total}, time={time.time()-t0:.1f}s', flush=True)
    return acc

# One-epoch smoke train
epochs = 1
print(f'Smoke train: epochs={epochs}, bs={bs}, train_n={len(train_ds)}, val_n={len(val_ds)}', flush=True)
best_acc = 0.0
t_start = time.time()
for epoch in range(epochs):
    model.train()
    t0 = time.time()
    running_loss = 0.0
    seen = 0
    for it, (xb, yb) in enumerate(train_loader):
        xb = xb.to(device, non_blocking=True, memory_format=torch.channels_last)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * yb.size(0)
        seen += yb.size(0)
        if (it+1) % 20 == 0:
            elapsed = time.time() - t0
            print(f'ep {epoch} it {it+1}/{math.ceil(len(train_ds)/bs)} loss {running_loss/seen:.4f} elapsed {elapsed:.1f}s', flush=True)
    train_loss = running_loss / max(1, seen)
    print(f'Epoch {epoch}: train_loss={train_loss:.4f} epoch_time={time.time()-t0:.1f}s total_elapsed={time.time()-t_start:.1f}s', flush=True)
    acc = evaluate(model, val_loader)
    best_acc = max(best_acc, acc)

print('Smoke training complete. Best val acc:', f'{best_acc:.4f}')
torch.cuda.empty_cache(); gc.collect()

Creating model: torchvision resnet50, weights IMAGENET1K_V2


Smoke train: epochs=1, bs=96, train_n=2048, val_n=512


/tmp/ipykernel_429/1375595590.py:74: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/tmp/ipykernel_429/1375595590.py:108: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


ep 0 it 20/22 loss 6.8466 elapsed 5.1s


Epoch 0: train_loss=6.8426 epoch_time=5.8s total_elapsed=5.8s


/tmp/ipykernel_429/1375595590.py:85: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Eval: acc=0.0098, n=512, time=1.2s


Smoke training complete. Best val acc: 0.0098


37501

In [9]:
# Stage A Extended: ConvNeXt-Tiny @256px, Mixup=0.3, EMA=0.99985, cosine w/ lr floor; resume to 12 epochs; 2-crop TTA inference
import math, time, gc, os, json, random
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as tvm
from torchvision.transforms import InterpolationMode
from timm.utils import ModelEmaV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert torch.cuda.is_available(), 'CUDA required for timely training'

class JsonImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        with Image.open(row['file_name']) as img:
            img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, int(row['label'])

# Transforms
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
img_size = 256
train_tfms = T.Compose([
    T.RandomResizedCrop(img_size, scale=(0.5, 1.0), interpolation=InterpolationMode.BICUBIC),
    T.RandomHorizontalFlip(0.5),
    T.TrivialAugmentWide(interpolation=InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(mean, std),
    T.RandomErasing(p=0.25, scale=(0.02, 0.12), ratio=(0.3, 3.3), value='random'),
])
val_tfms = T.Compose([
    T.Resize(int(img_size/0.875), interpolation=InterpolationMode.BICUBIC),
    T.CenterCrop(img_size),
    T.ToTensor(),
    T.Normalize(mean, std),
])

# Datasets & loaders
train_ds = JsonImageDataset(df_train, transform=train_tfms)
val_ds = JsonImageDataset(df_val, transform=val_tfms)
bs = 128
nw = min(8, os.cpu_count() or 4)
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=nw, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=nw, pin_memory=True, persistent_workers=True)
print(f'train_n={len(train_ds)} val_n={len(val_ds)} bs={bs} workers={nw}', flush=True)

# Model: ConvNeXt Tiny
weights = tvm.ConvNeXt_Tiny_Weights.IMAGENET1K_V1
model = tvm.convnext_tiny(weights=weights)
if isinstance(model.classifier, nn.Sequential) and isinstance(model.classifier[-1], nn.Linear):
    in_ch = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_ch, num_classes)
else:
    # Fallback in case of different head structure
    for name, m in list(model.named_modules())[::-1]:
        if isinstance(m, nn.Linear) and m.in_features > 0:
            setattr(model, name.split('.')[-1], nn.Linear(m.in_features, num_classes))
            break
model = model.to(device).to(memory_format=torch.channels_last)

# Optimizer, scheduler, loss, Mixup, EMA
base_lr = 1e-3
lr_peak = base_lr * (bs / 256.0)
lr_min = 1e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=lr_peak, weight_decay=0.05, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
scaler = torch.amp.GradScaler('cuda', enabled=True)
epochs = 12  # total target epochs
warmup_epochs = 1
steps_per_epoch = math.ceil(len(train_ds)/bs)
total_steps = epochs * steps_per_epoch
warmup_steps = warmup_epochs * steps_per_epoch

def cosine_lr(step):
    # returns multiplicative factor for lr between [lr_min/lr_peak, 1]
    if step < warmup_steps:
        return (step + 1) / max(1, warmup_steps)
    prog = (step - warmup_steps) / max(1, total_steps - warmup_steps)
    cos_val = 0.5 * (1 + math.cos(math.pi * prog))
    scaled = (lr_min / lr_peak) + (1 - (lr_min / lr_peak)) * cos_val
    return scaled

# Mixup utilities
mixup_alpha = 0.3
def mixup_batch(x, y, alpha=mixup_alpha):
    if alpha <= 0:
        return x, y, 1.0, None
    lam = float(np.random.beta(alpha, alpha)) if alpha > 0 else 1.0
    batch_size = x.size(0)
    index = torch.randperm(batch_size, device=x.device)
    y_a, y_b = y, y[index]
    x = x * lam + x[index] * (1.0 - lam)
    return x, (y_a, y_b), lam, index

# EMA
ema_decay = 0.99985
model_ema = ModelEmaV2(model, decay=ema_decay, device=device)

# Optional resume from best EMA checkpoint if exists
ckpt_path = Path('artifacts/ckpt_convnext_tiny_stageA_fast.pth')
best_acc = 0.0
if ckpt_path.exists():
    try:
        ckpt = torch.load(ckpt_path, map_location='cpu')
        model.load_state_dict(ckpt['state_dict'], strict=False)
        model_ema.module.load_state_dict(ckpt['state_dict'], strict=False)
        best_acc = float(ckpt.get('val_acc', 0.0))
        print(f'Resumed weights from {ckpt_path} (prev best_acc={best_acc:.4f})', flush=True)
    except Exception as e:
        print('Resume failed:', e, flush=True)

def evaluate(model_eval, loader):
    model_eval.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True, memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True):
                logits = model_eval(xb)
            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.numel()
    return correct / max(1, total)

# Train loop
t0_all = time.time()
step = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    seen = 0
    t0 = time.time()
    for it, (xb, yb) in enumerate(train_loader):
        lr_now = lr_peak * cosine_lr(step)
        for pg in optimizer.param_groups:
            pg['lr'] = lr_now
        step += 1
        xb = xb.to(device, non_blocking=True, memory_format=torch.channels_last)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        # Mixup
        xb_m, y_pair, lam, _ = mixup_batch(xb, yb, mixup_alpha)
        with torch.amp.autocast('cuda', enabled=True):
            logits = model(xb_m)
            if isinstance(y_pair, tuple):
                ya, yb2 = y_pair
                # Use criterion for both terms (respects label smoothing) - expert fix
                loss = lam * criterion(logits, ya) + (1.0 - lam) * criterion(logits, yb2)
            else:
                loss = criterion(logits, y_pair)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # EMA update
        model_ema.update(model)
        running_loss += loss.item() * yb.size(0)
        seen += yb.size(0)
        if (it + 1) % 100 == 0:
            print(f'ep {epoch} it {it+1}/{steps_per_epoch} loss {running_loss/max(1,seen):.4f} lr {lr_now:.6f} elapsed {time.time()-t0:.1f}s', flush=True)
    train_loss = running_loss / max(1, seen)
    # Evaluate EMA model
    val_acc = evaluate(model_ema.module, val_loader)
    print(f'Epoch {epoch}: train_loss={train_loss:.4f} val_acc={val_acc:.4f} epoch_time={time.time()-t0:.1f}s total_elapsed={time.time()-t0_all:.1f}s', flush=True)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'state_dict': model_ema.module.state_dict(), 'val_acc': best_acc}, ckpt_path)
        print(f'Saved new best to {ckpt_path} (acc={best_acc:.4f})', flush=True)

print(f'Training done. Best val_acc={best_acc:.4f}', flush=True)
del train_loader; torch.cuda.empty_cache(); gc.collect()

# Inference on test set with 2-crop TTA (orig + hflip) using EMA weights
with open('artifacts/idx_to_cat_id.json','r') as f:
    idx_to_cat_id = {int(k): int(v) for k,v in json.load(f).items()}
with open('test2019.json','r') as f:
    test_js = json.load(f)
test_images = test_js['images']
test_df = pd.DataFrame(test_images)
test_df['path'] = test_df['file_name']

class TestDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.tf = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        with Image.open(row['path']) as img:
            img = img.convert('RGB')
        return self.tf(img), int(row['id'])

test_ds = TestDataset(test_df, val_tfms)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=nw, pin_memory=True, persistent_workers=True)

# Load best EMA ckpt for inference
ckpt = torch.load(ckpt_path, map_location='cpu')
model = tvm.convnext_tiny(weights=None)
if isinstance(model.classifier, nn.Sequential) and isinstance(model.classifier[-1], nn.Linear):
    in_ch = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_ch, num_classes)
else:
    for name, m in list(model.named_modules())[::-1]:
        if isinstance(m, nn.Linear) and m.in_features > 0:
            setattr(model, name.split('.')[-1], nn.Linear(m.in_features, num_classes))
            break
model.load_state_dict(ckpt['state_dict'], strict=True)
model = model.to(device).to(memory_format=torch.channels_last).eval()

pred_rows = []
t_inf0 = time.time()
with torch.no_grad():
    for it, (xb, ids) in enumerate(test_loader):
        xb = xb.to(device, non_blocking=True, memory_format=torch.channels_last)
        with torch.amp.autocast('cuda', enabled=True):
            logits1 = model(xb)
            logits2 = model(torch.flip(xb, dims=[3]))  # horizontal flip
            logits = (logits1 + logits2) * 0.5
        preds = logits.argmax(1).detach().cpu().numpy()
        for img_id, p in zip(ids.tolist(), preds.tolist()):
            pred_rows.append((img_id, idx_to_cat_id[p]))
        if (it + 1) % 50 == 0:
            print(f'infer it {it+1}/{math.ceil(len(test_ds)/256)} elapsed {time.time()-t_inf0:.1f}s', flush=True)

sub = pd.DataFrame(pred_rows, columns=['image_id','category_id'])
sub = sub.sort_values('image_id').reset_index(drop=True)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with', len(sub), 'rows', flush=True)

torch.cuda.empty_cache(); gc.collect()

# Next: Stage B fine-tune @384px (4-5 epochs, lr~1e-4, mixup=0.1, RE p=0.1), 2-crop TTA.

train_n=230319 val_n=2680 bs=128 workers=8


Resumed weights from artifacts/ckpt_convnext_tiny_stageA_fast.pth (prev best_acc=0.6698)


/tmp/ipykernel_429/3138364984.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


ep 0 it 100/1800 loss 3.3352 lr 0.000028 elapsed 32.2s


ep 0 it 200/1800 loss 3.2771 lr 0.000056 elapsed 63.1s


...

/tmp/ipykernel_429/3138364984.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


infer it 50/126 elapsed 23.1s


infer it 100/126 elapsed 43.3s


Saved submission.csv with 32214 rows


52

In [10]:
# Validate and fix submission.csv header/types for Kaggle format
import pandas as pd
import numpy as np
from pathlib import Path

sub_fp = Path('submission.csv')
assert sub_fp.exists(), 'submission.csv not found'
sub = pd.read_csv(sub_fp)
print('Before:', sub.shape, sub.columns.tolist(), sub.dtypes.to_dict())

# Kaggle iNat2019 expects columns: id, category_id
if 'image_id' in sub.columns and 'id' not in sub.columns:
    sub = sub.rename(columns={'image_id': 'id'})

# Ensure correct columns and dtypes
assert set(sub.columns) == {'id','category_id'}, f'Unexpected columns: {sub.columns.tolist()}'
sub['id'] = sub['id'].astype(np.int64)
sub['category_id'] = sub['category_id'].astype(np.int64)
sub = sub.sort_values('id').reset_index(drop=True)

sub.to_csv(sub_fp, index=False)
print('After:', sub.shape, sub.columns.tolist(), sub.dtypes.to_dict())
print(sub.head().to_string(index=False))
print(sub.tail().to_string(index=False))

Before: (32214, 2) ['image_id', 'category_id'] {'image_id': dtype('int64'), 'category_id': dtype('int64')}
After: (32214, 2) ['id', 'category_id'] {'id': dtype('int64'), 'category_id': dtype('int64')}
 id  category_id
 20          167
 86          159
101          194
108          471
112          410
    id  category_id
265186          353
265188          902
265193          350
265194          712
265197          328


In [12]:
# Force submission header to ['image_id','category_id'] (some competitions expect this exact header)
import pandas as pd
from pathlib import Path

sub_fp = Path('submission.csv')
assert sub_fp.exists(), 'submission.csv not found'
sub = pd.read_csv(sub_fp)
print('Current columns:', sub.columns.tolist(), 'shape:', sub.shape, flush=True)
if 'id' in sub.columns and 'image_id' not in sub.columns:
    sub = sub.rename(columns={'id': 'image_id'})
assert set(sub.columns) == {'image_id','category_id'}, f'Unexpected columns: {sub.columns.tolist()}'
sub = sub[['image_id','category_id']].copy()
sub['image_id'] = sub['image_id'].astype('int64')
sub['category_id'] = sub['category_id'].astype('int64')
sub = sub.sort_values('image_id').reset_index(drop=True)
sub.to_csv(sub_fp, index=False)
print('Fixed columns:', sub.columns.tolist(), 'shape:', sub.shape, flush=True)
print(sub.head().to_string(index=False))
print(sub.tail().to_string(index=False))

Current columns: ['id', 'category_id'] shape: (32214, 2)


Fixed columns: ['image_id', 'category_id'] shape: (32214, 2)


 image_id  category_id
       20          167
       86          159
      101          194
      108          471
      112          410
 image_id  category_id
   265186          353
   265188          902
   265193          350
   265194          712
   265197          328


In [11]:
# Robust submission validator: ensure rows/ids match test2019.json and header ['id','category_id']
import json, pandas as pd, numpy as np
from pathlib import Path

sub_fp = Path('submission.csv')
assert sub_fp.exists(), 'submission.csv not found'
sub = pd.read_csv(sub_fp)
if 'image_id' in sub.columns and 'id' not in sub.columns:
    sub = sub.rename(columns={'image_id':'id'})
assert 'id' in sub.columns and 'category_id' in sub.columns, f'Unexpected cols: {sub.columns.tolist()}'

# Load test ids from JSON
with open('test2019.json','r') as f:
    test_js = json.load(f)
test_ids = pd.DataFrame({'id': [int(im['id']) for im in test_js['images']]})
test_ids = test_ids.sort_values('id').reset_index(drop=True)

# Check id coverage
sub_ids = set(sub['id'].astype(int).tolist())
test_id_set = set(test_ids['id'].tolist())
missing = test_id_set - sub_ids
extra = sub_ids - test_id_set
print(f'missing_in_sub={len(missing)} extra_in_sub={len(extra)} sub_n={len(sub)} test_n={len(test_ids)}', flush=True)

# Restrict to test ids, drop extras, align order exactly to test ids
sub = sub[['id','category_id']].copy()
sub['id'] = sub['id'].astype(np.int64)
sub['category_id'] = sub['category_id'].astype(np.int64)
sub = test_ids.merge(sub, on='id', how='left')
assert len(sub) == len(test_ids), 'Row count mismatch after align'
assert sub['category_id'].notna().all(), 'Found NaNs in category_id after align; predictions missing for some ids'

sub = sub[['id','category_id']].astype({'id':'int64','category_id':'int64'})
sub.to_csv(sub_fp, index=False)
print('Final submission:', sub.shape, sub.columns.tolist(), sub.dtypes.to_dict(), flush=True)
print(sub.head().to_string(index=False))
print(sub.tail().to_string(index=False))

missing_in_sub=0 extra_in_sub=0 sub_n=32214 test_n=32214


Final submission: (32214, 2) ['id', 'category_id'] {'id': dtype('int64'), 'category_id': dtype('int64')}


 id  category_id
 20          167
 86          159
101          194
108          471
112          410
    id  category_id
265186          353
265188          902
265193          350
265194          712
265197          328


In [13]:
# Convert submission.csv header to ['id','predicted'] (MLE-Benchmark/Kaggle variant expectation)
import pandas as pd
from pathlib import Path

sub_fp = Path('submission.csv')
assert sub_fp.exists(), 'submission.csv not found'
sub = pd.read_csv(sub_fp)
cols = sub.columns.tolist()
if 'image_id' in cols and 'id' not in cols:
    sub = sub.rename(columns={'image_id': 'id'})
if 'category_id' in sub.columns and 'predicted' not in sub.columns:
    sub = sub.rename(columns={'category_id': 'predicted'})
assert set(sub.columns) == {'id','predicted'}, f'Unexpected columns: {sub.columns.tolist()}'
sub['id'] = sub['id'].astype('int64')
sub['predicted'] = sub['predicted'].astype('int64')
sub = sub.sort_values('id').reset_index(drop=True)
sub.to_csv(sub_fp, index=False)
print('submission.csv fixed to columns:', sub.columns.tolist(), 'shape:', sub.shape, flush=True)
print(sub.head().to_string(index=False))
print(sub.tail().to_string(index=False))

submission.csv fixed to columns: ['id', 'predicted'] shape: (32214, 2)


 id  predicted
 20        167
 86        159
101        194
108        471
112        410
    id  predicted
265186        353
265188        902
265193        350
265194        712
265197        328
